In [1]:
include("SpinonStructure.jl")
include("BZMath.jl")
# include("PyrochloreGeometry.jl")
# import .PyrochloreGeometry as geom
using .BZmath
using .SpinonStructure
using StaticArrays
using BenchmarkTools
using Plots


In [2]:
@inline function gaussian(x,σ)
    local N = 1/√(2π)
    return N*exp(-0.5*(x/σ)^2)/σ
end

@inline function Lorentzian(x, Γ)
    local N = 1/π
    Γ /= 2
    return N*1. * Γ/(x^2 + Γ^2)
end

Lorentzian (generic function with 1 method)

In [3]:
A_dict = Dict(
    "0flux_big"=>load_A("gaugefiles/0_flux_big.gauge"),
    "0flux" => load_A("gaugefiles/0_flux_manual.gauge"),
    "πflux" => [ 0 0 π π ; 0 0 0 0; 0 0 π π; 0 0 0 0 ],
    "ππ00" =>  [ 0 0 0 π ; 0 0 0 0; 0 0 0 π; 0 0 0 0 ],
    "FF_even"=>load_A("gaugefiles/FF_even_pi18.gauge")
    )

high_symmetry_points = Dict(
    "\\Gamma"=> [0.,0.,0.],
    "X"=> [1.,0.,0.],
    "W"=> [1.,0.5,0.],
    "K"=> [0.75,0.75,0.],
    "L"=> [0.5,0.5,0.5],
    "U"=> [1.0, 0.25,0.25]
)

Dict{String, Vector{Float64}} with 6 entries:
  "U"       => [1.0, 0.25, 0.25]
  "W"       => [1.0, 0.5, 0.0]
  "X"       => [1.0, 0.0, 0.0]
  "\\Gamma" => [0.0, 0.0, 0.0]
  "L"       => [0.5, 0.5, 0.5]
  "K"       => [0.75, 0.75, 0.0]

In [4]:
"""
global_g must be a row vector
"""

function global_gauge(A_original, global_g)
    lat = geom.PyroFCC( round(Int, (size(A_original)[1]/4)^(1/3)) )
    
    A = copy(A_original) # make a copy
    # if local_g isa Vector{Float64}
    #     for (J, (t, Γ)) in enumerate(zip(lat.tetra_sites, local_g))
    #         eta =  1 if J <= length(lat.A_sites) else -1 
    #         for delta in geom.pyro
    #             A[geom.spin_idx(lat, t + eta*delta) += local_g
    #         end
    #     end
    # end
    return A_original .+ global_g
end
                
            

global_gauge (generic function with 1 method)

In [5]:
lat = geom.PyroFCC(3)
calc_fluxes(lat, A_dict["FF_even"])

108×4 Matrix{Float64}:
 2.09439  -0.698135  -0.698134  -0.698125
 2.09439  -0.698134  -0.698131  -0.698129
 2.09439  -0.698132  -0.69813   -0.698133
 2.09439  -0.698131  -0.698129  -0.698132
 2.09439  -0.698132  -0.698128  -0.698134
 2.0944   -0.698133  -0.698132  -0.698131
 2.09439  -0.698133  -0.698134  -0.698125
 2.09439  -0.698134  -0.698132  -0.698129
 2.0944   -0.698131  -0.698133  -0.698134
 2.09439  -0.698134  -0.698132  -0.698128
 2.0944   -0.698128  -0.698135  -0.698134
 2.0944   -0.698132  -0.698134  -0.698131
 2.09439  -0.698131  -0.698132  -0.698131
 ⋮                              
 2.0944   -0.698138  -0.698132  -0.698127
 2.0944   -0.698133  -0.698133  -0.69813
 2.09439  -0.69813   -0.698129  -0.698134
 2.09439  -0.698134  -0.69813   -0.698129
 2.0944   -0.69813   -0.698131  -0.698135
 2.09439  -0.698128  -0.698136  -0.69813
 2.09439  -0.69813   -0.698133  -0.69813
 2.0944   -0.698132  -0.698132  -0.698132
 2.09439  -0.698133  -0.698129  -0.698129
 2.0944   -0.698132  -0

# Calculating the Spectral Weight



In [6]:
# define static convenience types

const Vec3 = geom.Vec3;
const Vec3_F64 = geom.Vec3_F64;
################################################################################
### Calculating the spectral weight
################################################################################

"""
	specweight_at(q, Δ, sim)
	-> E, S

Calculates the contribution of kspace points(q ± Δ) to the Δ integral in 
`spectral_weight`. Let there be N tetrahedra in `sim`, i.e. N bands.
Returns: 
`E`, an (N, N) matrix of e1 + e2 energies corresponding to Dirac delta peaks
`S`, an (N, N) matrix of spectral weights giving the heights of these peaks
"""
function specweight_at(q::Vec3_F64, p::Vec3_F64, sim::SimulationParameters)
    p1 = q+p
    p2 = q-p
    
    E1, U1 = spinon_dispersion( p1, sim)
    E2, U2 = spinon_dispersion( p2, sim)
     
    # the form factor
    # W = (0.5+0im) .* [ 
    #     2 1 1 1; 
    #     1 2 1 1;
    #     1 1 2 1;
    #     1 1 1 2 
    # ]
    W = diagm(ones(ComplexF64, 4))

    for mu=1:4, nu=1:4
        # this is e^{-i p • (b_mu - b_nu)/2 ), factors of 2 cancel since pyro is 1/2 b
        W[mu,nu] *= exp(-1im* p'* (geom.pyro[mu] - geom.pyro[nu])) 
    end

    local f=length(sim.lat.tetra_sites)
    S = zeros(f,f)
    for mu=1:4, nu=1:4
        for (jA, rA) in enumerate(sim.lat.A_sites), (jpA, rpA) in enumerate(sim.lat.A_sites)
            jB = geom.tetra_idx(sim.lat, rA + geom.pyro[mu])
            jpB = geom.tetra_idx(sim.lat, rpA + geom.pyro[nu])

            # the "l" bit
            x1 = U1[jA, :] .* conj(U1[jpA, :]) ./ (2*E1) 
            # the "l'" bit
            x2 = U2[jB, :] .* conj(U2[jpB, :]) ./ (2*E2) 
            
            S += [ W[mu,nu]*ξ1*ξ2*exp(1im*(sim.A[jA,mu]-sim.A[jpA, nu])) for ξ2 in x2, ξ1 in x1 ]
        end
    end
    E = [e1 + e2 for e2 in E2, e1 in E1]::Matrix{Float64}
    return E, S
end





"""
    spectral_weight(q, Egrid, sim::SimulationParameters, 
						 nsample::Int=1000, grid_density::Int=1000
						 )

Calculates the spectral weight at point `q`.

This performs a Monte Carlo integral over the Brillouin zone.
`grid_density` sets the number of points in one dimension of the Brillouin zone.
`nsample` is the number of these k-points to sample.

"""
function spectral_weight(q::Vec3_F64, Egrid::Vector{Float64}, sim::SimulationParameters, 
						 integral_params::IntegrationParameters
						 )
    # cursed Monte Carlo integration
    num_flavours = div(length(sim.A),2)
    res = 0

    grid_density=integral_params.BZ_grid_density

    # p_vals = (@SMatrix rand((-grid_density,grid_density),integral_params.n_K_samples,3))*4π/8/grid_density
    p_vals = (1 .- 2 .*(@SMatrix rand(integral_params.n_K_samples,3)))*π/8
    
    Sqω = zeros(ComplexF64,size(Egrid))
    # perform an MC integral
    # dE = (Egrid[2]-Egrid[1])*broaden_factor

    bounds = [Inf, -Inf]
    
    for p in eachrow(p_vals)
        Enm, Snm = specweight_at(q, p, sim)
        Sqω += map(
            e-> sum( [S*Lorentzian(e - E, integral_params.broadening_dE) for (E,S) in zip(Enm,Snm)]),
                Egrid)
        bounds[1] = min(bounds[1], reduce(min,  Enm) ) 
        bounds[2] = max(bounds[2], reduce(max,  Enm) )
    end
    return Sqω, bounds
end

spectral_weight

In [7]:
function spectral_density_of_states(q::Vec3_F64, Egrid::Vector{Float64}, sim::SimulationParameters, 
						 integral_params::IntegrationParameters
						 )
    # cursed Monte Carlo integration
    num_flavours = div(length(sim.A),2)
    res = 0

    grid_density=integral_params.BZ_grid_density

    # p_vals = (@SMatrix rand((-grid_density,grid_density),integral_params.n_K_samples,3))*4π/8/grid_density
    p_vals = (1 .- 2 .*(@SMatrix rand(integral_params.n_K_samples,3)))*4π/8
    
    Sqω = zeros(ComplexF64,size(Egrid))
    

    bounds = [Inf, -Inf]
    
    # perform an MC integral
    for p in eachrow(p_vals)
        E1, U1 = spinon_dispersion( q+p, sim)
        E2, U2 = spinon_dispersion( q-p, sim)
        Enm = [e1 + e2 for e2 in E2, e1 in E1]
        Sqω += map( e-> sum([Lorentzian(e - E, integral_params.broadening_dE) for E in Enm]),
                Egrid)
        
        bounds[1] = min(bounds[1], reduce(min,  Enm) ) 
        bounds[2] = max(bounds[2], reduce(max,  Enm) )
    end
    return Sqω, bounds
end

using FastGaussQuadrature

function spectral_density_of_states_GL(q::Vec3_F64, Egrid::Vector{Float64}, sim::SimulationParameters, 
						 integral_params::IntegrationParameters
						 )
    num_flavours = div(length(sim.A),2)
    res = 0

    
    Sqω = zeros(ComplexF64,size(Egrid))

    bounds = [Inf, -Inf]
    ns = integral_params.n_K_samples
    
    ξ, w = gausslegendre( ns )
    
    px = ξ .* (π / 8 / sim.lat.L)
    py = ξ .* (π / 8 / sim.lat.L)
    pz = ξ .* (π / 8 / sim.lat.L)
    
    for ix=1:ns, iy=1:ns, iz=1:ns
        p = @SVector [ px[ix], py[iy], pz[iz] ]
        E1, U1 = spinon_dispersion( q+p, sim)
        E2, U2 = spinon_dispersion( q-p, sim)
        Enm = [e1 + e2 for e2 in E2, e1 in E1]
        Sqω += w[ix]*w[iy]*w[iz]*map( e-> sum([Lorentzian(e - E, integral_params.broadening_dE) for E in Enm]),
                Egrid)
        
        bounds[1] = min(bounds[1], reduce(min,  Enm) ) 
        bounds[2] = max(bounds[2], reduce(max,  Enm) )
    end
    return Sqω* (π / 8 / sim.lat.L)^3/8, bounds
end

spectral_density_of_states_GL (generic function with 1 method)

In [8]:
using JLD
using HDF5
using IJulia
using ProgressMeter

function simname(sim::SimulationParameters)
    return "output/SQW?name=$(sim.name)?Jpm=$(sim.Jpm)?B=$(sim.B).jld"
end
    
function save_SQW(;S::Matrix{ComplexF64}, bounds::Matrix{Float64}, Egrid::Vector{Float64}, BZ_path::BZPath,
        sim::SimulationParameters, 
        ip::IntegrationParameters)
    name = simname(sim)
    jldopen(name, "w") do file
        g1 = create_group(file, "integration_parameters")
        g1["n_K_samples"] = ip.n_K_samples
        g1["BZ_grid_density"] = ip.BZ_grid_density

        g = create_group(file, "physical_parameters")
        g["name"] = sim.name
        g["fluxes"] = sim.A
        g["Jpm"] = sim.Jpm
        g["B"] = sim.B
        g["lambda"] = sim.λ
        g["L"]=sim.lat.L

        d = create_group(file, "intensity") 
        d["S"] = S
        d["bounds"] = bounds
        # a list of K points, such that the I'th S slics corresponds to the I'th K point
        d["Q_list"] = path.K 
        d["tau"] = path.t
        d["ticks_tau"] = path.ticks_t
        d["ticks_label"] = path.ticks_label
        d["W"] = Egrid
    end
    return name
end
    


save_SQW (generic function with 1 method)

In [9]:
sim_0_Des = SimulationParameters("0flux_Des", A=A_dict["0flux"], Jpm=-0.046, B=zeros(3), nsample=1000, kappa=2.0)
sim_π_Des = SimulationParameters("πflux_Des", A=A_dict["πflux"], Jpm=1. /3, B=zeros(3), nsample=1000, kappa=2.0)
sim_π_Des_gauged = SimulationParameters("π_Des_gauged",A=global_gauge(sim_π_Des.A, [ 0. 0. 0. π/2 ]),
    Jpm = sim_π_Des.Jpm,
    B = sim_π_Des.B,
    kappa=2.
    )


LoadError: UndefVarError: `mean` not defined

In [10]:
## The 0-flux case
path = generate_path(high_symmetry_points, split("\\Gamma X W K \\Gamma L U W"), points_per_unit=30, K_units=2π/8)
E = reduce(vcat, map(k-> spinon_dispersion(k, sim_0_Des)[1], path.K )')

plot(path.t,E)
xticks!(path.ticks_t, path.ticks_label)
ylims!(0.,maximum(E))

LoadError: UndefVarError: `norm` not defined

In [11]:
## The π-flux case
path = generate_path(high_symmetry_points, split("\\Gamma X W K \\Gamma L U W"), points_per_unit=30, K_units=2π/8)
E = reduce(vcat, map(k-> spinon_dispersion(k, sim_π_Des_gauged)[1], path.K )')

plot(path.t,E)
xticks!(path.ticks_t, path.ticks_label)
ylims!(0.,maximum(E))

LoadError: UndefVarError: `norm` not defined

In [12]:
sim_ππ00 = SimulationParameters("ππ00", A=A_dict["ππ00"], Jpm=-0.05, B=0.3*[0,1,1]/sqrt(2), nsample=1000, kappa=2.0)
sim_FF_Even = SimulationParameters("FF", A=A_dict["FF_even"], Jpm=-0.05, B=0.3* [1.,1.,1.]/sqrt(3), nsample=1000, kappa=2.0)

LoadError: UndefVarError: `mean` not defined

In [13]:
## The ππ00-flux case
path = generate_path(high_symmetry_points, split("\\Gamma X W K \\Gamma L U W"), points_per_unit=30, K_units=2π/8)
E = reduce(vcat, map(k-> spinon_dispersion(k, sim_ππ00)[1], path.K )')

plot(path.t,E)
xticks!(path.ticks_t, path.ticks_label)
ylims!(0.,maximum(E))

LoadError: UndefVarError: `norm` not defined

In [14]:
## The ππ00-flux case
path = generate_path(high_symmetry_points, split("\\Gamma X W K \\Gamma L U W"), points_per_unit=30, K_units=2π/8)
E = reduce(vcat, map(k-> spinon_dispersion(k, sim_FF_Even)[1], path.K )')

plot(path.t,E)
xticks!(path.ticks_t, path.ticks_label)
ylims!(0.,maximum(E))

LoadError: UndefVarError: `norm` not defined

In [15]:
path = generate_path(high_symmetry_points, split("\\Gamma X W K \\Gamma L U W"), points_per_unit=10, K_units=2π/8)

function run_sim(sim::SimulationParameters, ip::IntegrationParameters, Egrid::Vector{Float64})
    
    num_K = length(path.K)
    
    S = zeros(ComplexF64, num_K, length(Egrid))
    bounds = zeros(Float64, num_K, 2)
    

    p = Progress(num_K)

    @time begin
        Threads.@threads for I = 1:num_K
            k = path.K[I]*0.5
            q = SVector(k[1], k[2], k[3])
            S[I, :], bounds[I,:] = spectral_weight(q, Egrid, sim, ip)
            # S[I, :], bounds[I,:] = spectral_density_of_states_GL(q, Egrid, sim, ip)
            next!(p)
        end
    end
    finish!(p)

    # save the data
    save_SQW(S=S, bounds=bounds, BZ_path=path, Egrid=collect(Egrid), sim=sim, ip=ip)
end

LoadError: UndefVarError: `norm` not defined

In [16]:
ip_GL = IntegrationParameters(n_K_samples=8,BZ_grid_density=50,broadening_dE=0.2)

ip_fast = IntegrationParameters(n_K_samples=100,BZ_grid_density=50,broadening_dE=0.2)
ip_slow = IntegrationParameters(n_K_samples=1000,BZ_grid_density=500,broadening_dE=0.2)
ip_very_slow = IntegrationParameters(n_K_samples=10000,BZ_grid_density=500,broadening_dE=0.2)


IntegrationParameters(10000, 500, 0.2)

In [17]:
Egrid = collect(range(0,2,100));
res_0_Des = run_sim(sim_0_Des, ip_slow, Egrid)
Egrid = collect(range(0,3,100));
res_π_Des = run_sim(sim_π_Des_gauged, ip_slow, Egrid)

LoadError: UndefVarError: `run_sim` not defined

In [18]:
Egrid = collect(range(0,2,100));
res_ππ00 = run_sim(sim_ππ00, ip_slow, Egrid)

LoadError: UndefVarError: `run_sim` not defined

In [19]:
Egrid = collect(range(0,2,100));
res_FF_Even = run_sim(sim_FF_Even, ip_fast, Egrid)

LoadError: UndefVarError: `run_sim` not defined

In [20]:
function plot_dispersion(data)
    d = data["intensity"]
    p = heatmap(d["tau"],d["W"],real.(d["S"])')
    plot!(d["tau"], d["bounds"], linecolor=:white)
    xticks!(d["ticks_tau"],d["ticks_label"])
    plot!(legend=nothing)

    savefig("output/$(data["physical_parameters"]["name"]).pdf")
    title!(data["physical_parameters"]["name"])
    return p
end

plot_dispersion (generic function with 1 method)

In [21]:
?savefig

search: savefig StackOverflowError



```
savefig([plot,] filename)
```

Save a Plot (the current plot if `plot` is not passed) to file. The file type is inferred from the file extension. All backends support png and pdf file types, some also support svg, ps, eps, html and tex.


In [22]:
plot_dispersion( load(simname(sim_π_Des_gauged) ) )

LoadError: UndefVarError: `sim_π_Des_gauged` not defined

In [23]:
data = load(simname(sim_0_Des) )
d = data["intensity"]
heatmap(d["tau"],d["W"],real.(d["S"])')
plot!(d["tau"], d["bounds"], linecolor=:white)
plot!(legend=nothing)
xticks!(d["ticks_tau"],d["ticks_label"])
savefig("output/$(data["physical_parameters"]["name"]).pdf")
title!(data["physical_parameters"]["name"])

LoadError: UndefVarError: `sim_0_Des` not defined

In [24]:
data = load(simname(sim_ππ00))
d = data["intensity"]
heatmap(d["tau"],d["W"],real.(d["S"])')
plot!(d["tau"], d["bounds"], linecolor=:white)
xticks!(d["ticks_tau"],d["ticks_label"])
savefig("output/$(data["physical_parameters"]["name"]).pdf")
title!(data["physical_parameters"]["name"])

LoadError: UndefVarError: `sim_ππ00` not defined

In [25]:
res_FF_Even = run_sim(sim_FF_Even, ip_fast, Egrid)

LoadError: UndefVarError: `run_sim` not defined

In [26]:
data = load(simname(sim_FF_Even))
d = data["intensity"]
heatmap(d["tau"],d["W"],real.(d["S"])')
plot!(d["tau"], d["bounds"], linecolor=:white)
xticks!(d["ticks_tau"],d["ticks_label"])
savefig("output/$(data["physical_parameters"]["name"]).pdf")
title!(data["physical_parameters"]["name"])

LoadError: UndefVarError: `sim_FF_Even` not defined

In [27]:
using LaTeXStrings
plot_font = "Computer Modern"
default(
    fontfamily=plot_font,
    linewidth=2, 
    framestyle=:box, 
    label=nothing, 
    grid=false
)

function get_FCC_WS_edges()

    squareX = [
        [1.,  0.5,    0],
        [1.,  0.,  -0.5],
        [1., -0.5,    0],
        [1.,  0.,   0.5],
        [1.,  0.5,    0]
    ]
    neg_squareX= map(x->x.*[-1,1,1], squareX)

    hexa_111 = [
        [1. , 0.5, 0.],
        [0.5, 1. , 0.],
        [0. , 1., 0.5],
        [0., 0.5 , 1.],
        [0.5, 0. , 1.],
        [1., 0., 0.5]
    ]

    square_faces = [
        squareX,
        neg_squareX,
        circshift.(squareX,1),
        circshift.(neg_squareX,1),
        circshift.(squareX,2),
        circshift.(neg_squareX,2)
    ]

    # unused right now, but might come in handy some day
    # hexagons = [
    #     hexa_111,
    #     map(x->x.*[-1,-1,1], hexa_111),
    #     map(x->x.*[-1,1,-1], hexa_111),
    #     map(x->x.*[1,-1,-1], hexa_111),
    #     map(x->x.*[-1,1,1], hexa_111),
    #     map(x->x.*[1,-1,1], hexa_111),
    #     map(x->x.*[1,1,-1], hexa_111),
    #     map(x->x.*[-1,-1,-1], hexa_111)
    # ]

    e0 = [
        [ [ 1.,  0.5, 0.], [ 0.5,  1., 0.] ],
        [ [ 1., -0.5, 0.], [ 0.5, -1., 0.] ],
        [ [-1., -0.5, 0.], [-0.5, -1., 0.] ],
        [ [-1.,  0.5, 0.], [-0.5,  1., 0.] ],
    ]
    edges = copy(e0)
    append!(edges, map( l -> circshift.(l,1), e0))
    append!(edges, map( l -> circshift.(l,2), e0))

    return vcat(square_faces, edges)
end
    
p = plot(camera=(80,15))

for f in get_FCC_WS_edges()
    ff = reduce(hcat, f)
    plot!( ff[1,:],ff[2,:],ff[3,:], linecolor="black", linewidth=.5 )
end

X = []
for (label, x) in high_symmetry_points
    push!(labels, label)
    push!(X, x)
    annotate!(x[1],x[2],x[3],text("\$$(label)\$",:left,:bottom))
end
X= reduce(hcat, X)
plot!(X[1,:],X[2,:],X[3,:], seriestype=:scatter)

kk = reduce(hcat, path.K)'*8/2/π
plot!(kk[:,1],kk[:,2],kk[:,3],linecolor=:red)
display(p)

LoadError: UndefVarError: `labels` not defined